In [1]:
MODEL = "llama3.2"

In [2]:
import pydantic
print(pydantic.__version__)


2.10.6


In [11]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)
# model.invoke("where are you prividing your response from")


In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./paper-crash-detection.pdf")
pages = loader.load_and_split()

In [5]:
pages

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2024-04-30T11:04:15+05:30', 'author': 'Microsoft account', 'moddate': '2024-04-30T11:04:15+05:30', 'source': './paper-crash-detection.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content="Journal of IoT in Social, Mobile, Analytics, and Cloud (ISSN: 2582-1369)  \nwww.irojournals.com/iroismac/    \nJournal of IoT in Social, Mobile, Analytics, and Cloud, March 2024, Volume 6, Issue 1, Pages 54-66 54 \nDOI: https://doi.org/10.36548/jismac.2024.1.005 \nReceived: 05.03.2024, received in revised form: 06.04.2024, accepted: 18.04.2024, published: 30.04.2024 \n© 2024 Inventive Research Organization. This is an open access article under the Creative Commons Attribution-NonCommercial International (CC BY-NC 4.0) License \nDrowsiness and Crash Detection Mobile \nApplication for Vehicle’s Safety \nNabaraj Subedi1, Nirajan Paudel2, Manish Chhetri3, Sudarshan \nAcharya4, Nabin 

In [22]:
from langchain.prompts import PromptTemplate

template = """
Answer the questions based on the context below. If you cannot answer the question given, just reply I don't know

Context: {context}
Question: {question}

"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="", question=""))


Answer the questions based on the context below. If you cannot answer the question given, just reply I don't know

Context: 
Question: 




In [23]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [24]:
chain = prompt | model | parser

In [25]:
chain.invoke(
    {
        "context":"nepal", 
        "question":"what is the country name"
    }
)

'The country name in Nepal is "Nepal".'

In [26]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embeddings
)

In [27]:
vectorstore

In [28]:
retreiver = vectorstore.as_retriever()

In [29]:
retreiver.invoke("GPS module")

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2024-04-30T11:04:15+05:30', 'author': 'Microsoft account', 'moddate': '2024-04-30T11:04:15+05:30', 'source': './paper-crash-detection.pdf', 'total_pages': 13, 'page': 5, 'page_label': '6'}, page_content="Nabaraj Subedi, Nirajan Paudel, Manish Chhetri, Sudarshan Acharya, Nabin Lamichhane  \nJournal of IoT in Social, Mobile, Analytics, and Cloud, March 2024, Volume 6, Issue 1 59 \n \nFigure 3. Crash Detection Flow Chart \n3.3 Components Used \na.   Arduino UNO  \nThe Arduino Uno, built around the ATmega328P microcontroller, receives velocity and \ntilt values from the accelerometer [7]. It verifies if these values meet predefined conditions \nbefore proceeding with further operations. \nb.   MPU 6050 \nIn this study, a 10 DOF IMU Sensor, capable of measuring 3 -axis accelerometer data, \nis utilized to detect changes in a vehicle's orientation, indicative of potential accidents. To

In [30]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retreiver,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

In [31]:
chain.invoke({"question":"What is the pdf about?"})

"The PDF appears to be an academic paper or research document related to the development and implementation of an accident detection system for vehicles using IoT technology (Internet of Things). The system includes sensors such as accelerometers, GPS, and a GSM module to detect crashes and drowsiness in drivers. The paper discusses various aspects of the system's design, methodology, and potential challenges that need to be addressed."